# Setup

In [1]:
import pandas as pd
from zipfile import ZipFile
import os
import csv
import janitor
import pandas_gbq
import numpy as np
import re
import io
from google.cloud import bigquery
from google.oauth2 import service_account
import sqlite3

In [2]:
service_path = "\\Users\\aidan\\OneDrive\\Desktop\\MSBA_F22\\ADA\\"
service_file = 'placeholder-9bb4f6c64d82.json'   
gbq_proj_id = 'placeholder' 
 
private_key =service_path + service_file
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

bigquerywagers:Wedge_B_level
bigquerywagers:Wedge_Wagers
bigquerywagers:Wedge_clean
bigquerywagers:dram_shop
bigquerywagers:umt_msba_aw


# Query 1: Sales by Date by Hour

In [3]:
query = """
    SELECT 
    Extract(Date from datetime) as date,
    Extract(HOUR FROM datetime) as hour,
    sum(total) as total_spend,
    count(distinct(concat(trans_no,Extract(Date from datetime),register_no,emp_no))) AS num_trans
    FROM `placeholder.Wedge_Wagers.tran*` 
    GROUP BY Hour, date
    ORDER BY total_spend DESC
"""

Sales_Date_Hour = pandas_gbq.read_gbq(query,project_id = 'placeholder')

Sales_Date_Hour.to_csv('sales_date_hour.csv', index = False, encoding = 'utf-8')

Downloading: 100%|██████████| 39420/39420 [00:01<00:00, 20016.20rows/s]


# Query 2: Sales by Owner x Year x Month

In [4]:
query = """
    SELECT 
    card_no,
    EXTRACT(YEAR FROM datetime) AS `year`,
    EXTRACT(MONTH FROM datetime) AS `month`,
    sum(abs(total)) AS `sales`,
    COUNT(DISTINCT((CONCAT(CAST(EXTRACT(DATE from datetime)AS STRING),CAST(register_no AS String), CAST(trans_no AS String),CAST(emp_no AS String))))) AS `transactions`,
    sum(CASE WHEN trans_status in ('V','R') then -1 else 1 end) AS `items`
    FROM `bigquerywagers.Wedge_Wagers.transArchiv*`
    GROUP BY card_no,year, month
    ORDER BY year, month ASC
"""

owner_year_month = pandas_gbq.read_gbq(query,project_id = 'placeholder')

owner_year_month.to_csv('owner_year_month.csv', index = False, encoding = 'utf-8')

Downloading: 100%|██████████| 810208/810208 [00:31<00:00, 25450.85rows/s]


# Query 3: Sales by desc x year x month

In [11]:
threequery = """
    SELECT al.department as `department_number`,
    dl.dept_name as `department_name`,
    al.upc AS `upc`,
    al.description AS `description`,
    EXTRACT(YEAR FROM datetime) AS `year`,
    EXTRACT(MONTH FROM datetime) AS `month`,
    sum(abs(al.total)) AS `sales`,
    COUNT(DISTINCT((CONCAT(CAST(EXTRACT(DATE from datetime)AS STRING),CAST(al.register_no AS String), CAST(al.trans_no AS String),CAST(al.emp_no AS String))))) AS `transactions`,
    sum(CASE WHEN trans_status in ('V','R') then -1 else 1 end) AS `items`
    FROM `bigquerywagers.Wedge_Wagers.transArchiv*` as al
    INNER JOIN `bigquerywagers.Wedge_Wagers.depLook` as dl on al.department = dl.department
    GROUP BY al.description, year, month, upc, al.department, dept_name
    ORDER BY year, month ASC
"""

desc_year_month = pandas_gbq.read_gbq(threequery,project_id = 'placeholder')

desc_year_month.to_csv('desc_year_month.csv', index = False, sep = "\t", encoding = 'utf-8')

Downloading: 100%|██████████| 1214098/1214098 [01:18<00:00, 15502.96rows/s]


# SQLite Upload

In [12]:
db = sqlite3.connect("wedge_wagers.db")
cur = db.cursor()

#Create and fill First
cur.execute('''DROP TABLE IF EXISTS sales_date_hour''')
cur.execute('''CREATE TABLE sales_date_hour (
    date DATE, 
    hour INTEGER, 
    total_spend REAL,
    num_trans REAL)''')

input_list = []
with open("sales_date_hour.csv",'r') as infile :
    next(infile)

    for line in infile :
        line = line.strip().split(",")
        input_list.append(line)

cur.executemany('''INSERT INTO sales_date_hour(date,hour,total_spend,num_trans) 
                   VALUES (?,?,?,?)''',input_list)

#Create and fill Second
cur.execute('''DROP TABLE IF EXISTS owner_year_month''')
cur.execute('''CREATE TABLE owner_year_month (
    card_no INTEGER, 
    year INTEGER, 
    month INTEGER,
    sales REAL,
    transactions REAL,
    items INTEGER)''')

oym_list = []
with open("owner_year_month.csv",'r') as infile :
    next(infile)

    for line in infile :
        line = line.strip().split(",")
        oym_list.append(line)

cur.executemany('''INSERT INTO owner_year_month(card_no,year,month,sales,transactions,items) 
                   VALUES (?,?,?,?,?,?)''',oym_list)

#Create and fill Third
cur.execute('''DROP TABLE IF EXISTS desc_year_month''')
cur.execute('''CREATE TABLE desc_year_month (
    department_number INTEGER, 
    department_name TEXT, 
    upc INTEGER,
    description TEXT,
    year INTEGER,
    month INTEGER,
    sales REAL,
    transactions REAL,
    items INTEGER)''')

dym_list = []
with open("desc_year_month.csv",'r') as infile :
    next(infile)

    for line in infile :
        line = line.strip().split("\t")
        
        
        dym_list.append(line)
        

cur.executemany('''INSERT INTO desc_year_month(department_number,department_name,upc,description,year,month,sales,transactions,items) 
                   VALUES (?,?,?,?,?,?,?,?,?)''',dym_list)

db.commit()